In [18]:
import casadi as cs
import rbdl
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
import numpy as np
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
import pybullet as pb


root = 'base_link'
tip = 'wrist_3_link'
path_to_urdf = '/home/lmjohann/urdf2casadi/examples/urdf/ur5_rbdl.urdf'

#get robot models

#kdl
ok, ur_tree = kdlurdf.treeFromFile(path_to_urdf)
ur_chain = ur_tree.getChain(root,tip)

#rbdl
urmodel = rbdl.loadModel(path_to_urdf)

#u2c
ur5 = u2c.URDFparser()
robot_desc = ur5.from_file(path_to_urdf)

#pybullet
sim = pb.connect(pb.DIRECT)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True)
pb.setGravity(0, 0, -9.81)

#joint info
jointlist, names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = ur5.get_n_joints(root, tip)


#declarations

#kdl
q_kdl = kdl.JntArray(n_joints)
gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
g_kdl = kdl.JntArray(n_joints)
M_kdl = kdl.JntSpaceInertiaMatrix(n_joints)

#u2c & pybullet
q = [None]*n_joints
qdot = [None]*n_joints
zeros_pb = [None]*n_joints
M_sym = ur5.get_inertia_matrix_crba(root, tip)

#rbdl
q_np = np.zeros(n_joints)
M_rbdl = (n_joints, n_joints)
M_rbdl = np.zeros(M_rbdl)

#error declarations
error_kdl_rbdl = np.zeros((n_joints, n_joints))
error_kdl_u2c = np.zeros((n_joints, n_joints))
error_rbdl_u2c = np.zeros((n_joints, n_joints))
error_pb_u2c = np.zeros((n_joints, n_joints))


def u2c2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()

def list2np(asd):
    return np.asarray(asd)

n_itr = 10
for i in range(n_itr):
    for j in range(n_joints):
        q[j] = (q_max[j] - q_min[j])*np.random.rand()-(q_max[j] - q_min[j])/2
        q_kdl[j] = q[j]
        q_np[j] = q[j]


    rbdl.CompositeRigidBodyAlgorithm(urmodel, q_np, M_rbdl)
    kdl.ChainDynParam(ur_chain, gravity_kdl).JntToMass(q_kdl, M_kdl)
    M_pb = pb.calculateMassMatrix(pbmodel, q)
    M_u2c = M_sym(q)
    
for row_idx in range(n_joints):
    for col_idx in range(n_joints):
        error_kdl_u2c[row_idx][col_idx] += np.absolute(M_kdl[row_idx,col_idx] - u2c2np(M_u2c[row_idx, col_idx]))
        error_rbdl_u2c[row_idx][col_idx] += np.absolute((M_rbdl[row_idx,col_idx]) - u2c2np(M_u2c[row_idx, col_idx]))
        error_pb_u2c[row_idx][col_idx] += np.absolute((M_pb[row_idx][col_idx]) - u2c2np(M_u2c[row_idx, col_idx]))
        error_kdl_rbdl[row_idx][col_idx] += np.absolute((M_rbdl[row_idx,col_idx]) - list2np(M_kdl[row_idx, col_idx]))

    
print M_u2c[0,0]
print M_rbdl[0,0]
print M_kdl[0,0]
print M_pb[0][0]

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/urdf/ur5_rbdl.urdf
0.989591
0.9895905225427196
0.989590522541
1.56026081922


In [15]:
sum_error_kdl_rbdl = 0
sum_error_kdl_u2c = 0
sum_error_rbdl_u2c = 0
sum_error_pb_u2c = 0


for row in range(n_joints):
    for col in range(n_joints):
        sum_error_kdl_rbdl += error_kdl_rbdl[row][col]
        sum_error_kdl_u2c += error_kdl_u2c[row][col]
        sum_error_rbdl_u2c += error_rbdl_u2c[row][col]
        sum_error_pb_u2c += error_pb_u2c[row][col]

print "\nSum of errors KDL vs. RBDL for", n_itr, "iterations:\n", sum_error_kdl_rbdl
print "\nSum of errors KDL vs. U2C for", n_itr, "iterations:\n", sum_error_kdl_u2c
print "\nSum of errors RBDL vs. U2C for", n_itr, "iterations:\n",sum_error_rbdl_u2c
print "\nSum of errors pybullet vs. U2C for", n_itr, "iterations:\n", sum_error_pb_u2c



Sum of errors KDL vs. RBDL for 10 iterations:
4.7394575243553394e-11

Sum of errors KDL vs. U2C for 10 iterations:
2.817407765420832e-15

Sum of errors RBDL vs. U2C for 10 iterations:
4.739331258170332e-11

Sum of errors pybullet vs. U2C for 10 iterations:
1.435742481588182


In [26]:
import casadi as cs
import rbdl
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
import numpy as np
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
import pybullet as pb


root = 'base_link'
tip = 'wrist_3_link'
path_to_urdf = '/home/lmjohann/urdf2casadi/examples/urdf/ur5_rbdl.urdf'

#get robot models

#kdl
ok, ur_tree = kdlurdf.treeFromFile(path_to_urdf)
ur_chain = ur_tree.getChain(root,tip)

#rbdl
urmodel = rbdl.loadModel(path_to_urdf)

#u2c
ur5 = u2c.URDFparser()
robot_desc = ur5.from_file(path_to_urdf)

#pybullet
sim = pb.connect(pb.DIRECT)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True)
pb.setGravity(0, 0, -9.81)

#joint info
jointlist, names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = ur5.get_n_joints(root, tip)


#declarations

#kdl
q_kdl = kdl.JntArray(n_joints)
qdot_kdl = kdl.JntArray(n_joints)
gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
C_kdl = kdl.JntArray(n_joints)
g_kdl = kdl.JntArray(n_joints)

#u2c & pybullet
q = [None]*n_joints
qdot = [None]*n_joints
zeros_pb = [None]*n_joints
gravity_u2c = [0, 0, -9.81]
g_sym = ur5.get_gravity_rnea(root, tip, gravity_u2c)
C_sym = ur5.get_coriolis_rnea(root, tip)


#rbdl
q_np = np.zeros(n_joints)
qdot_np = np.zeros(n_joints)
qddot_np = np.zeros(n_joints)
g_rbdl = np.zeros(n_joints)
C_rbdl = np.zeros(n_joints)
zeros_rbdl = np.zeros(n_joints)



error_kdl_rbdl = np.zeros(n_joints)
error_kdl_u2c = np.zeros(n_joints)
error_rbdl_u2c = np.zeros(n_joints)
error_pb_u2c = np.zeros(n_joints)


def u2c2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()

def list2np(asd):
    x = []
    for i in range(n_joints):
        x.append(asd[i])
    return np.asarray(x)

n_itr = 10
for i in range(n_itr):
    for j in range(n_joints):
        q[j] = (q_max[j] - q_min[j])*np.random.rand()-(q_max[j] - q_min[j])/2
        q_kdl[j] = q[j]
        q_np[j] = q[j]

        qdot[j] = 0.#(q_max[j] - q_min[j])*np.random.rand()-(q_max[j] - q_min[j])/2
        qdot_kdl[j] = qdot[j]
        qdot_np[j] = qdot[j]
        
        zeros_pb[j] = 0.


    rbdl.NonlinearEffects(urmodel, q_np, qdot_np, C_rbdl)
    #rbdl.NonlinearEffects(urmodel, q_np, qdot_np, C_rbdl)
    kdl.ChainDynParam(ur_chain, gravity_kdl).JntToGravity(q_kdl, g_kdl)
    kdl.ChainDynParam(ur_chain, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, C_kdl)

    C_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, zeros_pb)

    g_u2c = g_sym(q)
    C_u2c = C_sym(q, qdot)
    
nle_u2c = g_u2c[0] + C_u2c[0]
nle_kdl = C_kdl[0] + g_kdl[0]


print g_u2c, g_kdl
print C_u2c, C_kdl

print nle_u2c, nle_kdl

print C_rbdl
print C_pb

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/urdf/ur5_rbdl.urdf
[-1.49483, 17.8097, 12.7762, -0.786384, 0.152348, 0]  -1.49483
  17.8097
  12.7762
-0.786384
 0.152348
        0
[0, 0, 0, 0, 0, 0] 0
0
0
0
0
0
-1.49483 -1.49482906745
[-1.49482907 17.80971792 12.77615598 -0.78638431  0.15234798  0.        ]
(-1.4948298138844045, 17.809717871033772, 12.776156091922994, -0.7863843372173319, 0.15234797469922237, 0.0)
